In [6]:
import pandas as pd
import numpy as np

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

import os

## Ignore warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
# np.warnings.filterwarnings('ignore')

os.chdir('..')
from maps.maps import fine_linktype_map
from maps.maps import status_map
from maps.maps import resolution_map
os.chdir('/informatik2/mobis/home/lueders/Desktop/Forschung/Research/DISS/SotP')
os.getcwd()

'/informatik2/mobis/home/lueders/Desktop/Forschung/Research/DISS/SotP'

In [7]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 
           'JiraEcosystem', 'MariaDB', 'Mindville', 'Mojang', 'MongoDB', 
           'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']

In [8]:
def load_cleaned_links(source):
    #Loading Issues
    filename = '../data/cleaned_links_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    link_df['maptype'] = link_df['linktype'].map(fine_linktype_map)
    
    return link_df

def load_detailed_issues(source):
    #Loading Issues
    filename = '../data/detailed_issues_'+source.lower()+'.csv'
    issue_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=',')
    
    return issue_df

def load_link_hist(source):
    #Loading Links
    filename =  '../data/hist_link_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=',')
    
    # IntelDAOS Epic
    # Jira Epicv
    # MariaDB Parent Link
    # Mojang Linked
    # RedHat User Story, Parent Link
    if source == 'IntelDAOS' or source == 'Jira':
        link_df = link_df[link_df['typeofchange']!='Epic']
    elif source == 'MariaDB':
        link_df = link_df[link_df['typeofchange']!='Parent Link']
    elif source == 'Mojang':
        link_df = link_df[link_df['typeofchange']!='Linked']
    elif source == 'RedHat':
        link_df = link_df[link_df['typeofchange']!='Parent Link']
        link_df = link_df[link_df['typeofchange']!='User Story']
        
    if link_df['maintainer'].isna().values.any():
        link_df['maintainer'] = link_df['displayName']
    link_df['maintainer'] = link_df['maintainer'].astype('string')
    
    return link_df

In [ ]:
link_dict = {}
issue_dict = {}
link_hist_dict = {}

for s in SOURCES:
    print(s.upper())
    
    links = load_cleaned_links(s)
    link_dict[s] = links
    
    link_hist = load_link_hist(s)
    link_hist_dict[s] = link_hist
    
    issues = load_detailed_issues(s)
    issue_dict[s] = issues

APACHE


In [ ]:
weird = []

for s in SOURCES:
    try:
        issues = issue_dict[s]
        try:
            issues['update'] = pd.to_datetime(issues['updated'], format='%Y-%m-%d')
        except:
            issues['updated2'] = issues['updated'].apply(lambda x: x[:10])
            try:
                issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d')
            except:
                issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d', errors='coerce')
                issues = issues[issues['update'].notna()]
        issues['year'] = issues['update'].dt.year
        issues['date'] = issues['update'].dt.date
    except:
        print(s + " why no work")
        weird.append(s)
    issue_dict[s] = issues
    
for s in weird:
    issues = issue_dict[s]
    issues['updated2'] = issues['updated'].apply(lambda x: x[:10])
    issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d', errors='coerce')
    issues['year'] = issues['update'].dt.year
    issues['date'] = issues['update'].dt.date
    issue_dict[s] = issues

In [ ]:
weird = []

for s in SOURCES:
    try:
        issues = link_hist_dict[s]
        try:
            issues['update'] = pd.to_datetime(issues['timestamp'], format='%Y-%m-%d')
        except:
            issues['updated2'] = issues['timestamp'].apply(lambda x: x[:10])
            try:
                issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d')
            except:
                issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d', errors='coerce')
                issues = issues[issues['update'].notna()]
        issues['year'] = issues['update'].dt.year
        issues['date'] = issues['update'].dt.date
    except:
        print(s + " why no work")
        weird.append(s)
    link_hist_dict[s] = issues
    
for s in weird:
    issues = link_hist_dict[s]
    issues['updated2'] = issues['timestamp'].apply(lambda x: x[:10])
    issues['update'] = pd.to_datetime(issues['updated2'], format='%Y-%m-%d', errors='coerce')
    issues['year'] = issues['update'].dt.year
    issues['date'] = issues['update'].dt.date
    link_hist_dict[s] = issues

In [ ]:
project_activity = pd.DataFrame(columns=['Repository', 'Project', 'last year active'])

for s in SOURCES:
    temp = issue_dict[s]
    for p in temp.projectid.unique():
        project_activity.loc[len(project_activity)] = [s, p, temp[temp['projectid']==p]['year'].max()]

maintainer_activity = pd.DataFrame(columns=['Repository', 'Maintainer', 'last year active'])

for s in SOURCES:
    temp = link_hist_dict[s]
    for p in temp.maintainer.unique():
        maintainer_activity.loc[len(maintainer_activity)] = [s, p, temp[temp['maintainer']==p]['year'].max()]


In [ ]:
project_activity['last year active'].value_counts()

In [ ]:
848/project_activity['last year active'].value_counts().sum()

In [ ]:
maintainer_activity['last year active'].value_counts()

In [ ]:
maintainer_all = {}
maintainer_5 = {}
maintainer_2 = {}
project_all = {}
project_5 = {}
project_2 = {}

mp_overview = pd.DataFrame(columns=['Repository', 'Total Projects', 'Projects5', 'Projects2', 
                                    'Total Maintainers', 'Maintainer5', 'Maintainer2'])

for s in SOURCES:
    print(s)
    temp_m = maintainer_activity[maintainer_activity['Repository']==s]
    maintainer_all[s] = set(temp_m['Maintainer'].values)
    temp_p = project_activity[project_activity['Repository']==s]
    project_all[s] = set(temp_p['Project'].values)
    x5 = temp_p['last year active'].max()-5
    x2 = temp_p['last year active'].max()-2
    temp_m5 = temp_m[temp_m['last year active']>=x5]
    temp_m2 = temp_m[temp_m['last year active']>=x2]
    temp_p5 = temp_p[temp_p['last year active']>=x5]
    temp_p2 = temp_p[temp_p['last year active']>=x2]
    maintainer_5[s] = set(temp_m5['Maintainer'].values)
    maintainer_2[s] = set(temp_m2['Maintainer'].values)
    project_5[s] = set(temp_p5['Project'].values)
    project_2[s] = set(temp_p2['Project'].values)
    
    mp_overview.loc[len(mp_overview)] = [s, len(temp_p), len(temp_p5), len(temp_p2),
                                         len(temp_m),len(temp_m5), len(temp_m2)]
mp_overview.to_csv('properties/mp_overview.csv', encoding='utf-8', index=True, sep=',')

mp_overview

In [ ]:
print(mp_overview.set_index('Repository').to_latex())

In [ ]:
mp_overview.sum()

In [ ]:
import re

for s in SOURCES:
    print(s)
    lh = link_hist_dict[s]
    lh.fillna(" ", inplace=True)
    lh.reset_index(drop=True, inplace=True)

    lh['to_1_len'] = lh['to'].apply(lambda x: len(str(x).split()))
    lh['from_1_len'] = lh['from'].apply(lambda x: len(str(x).split()))

    lh['to_2_len'] = lh['other_issue_to'].apply(lambda x: len(str(x).split()))
    lh['from_2_len'] = lh['other_issue_from'].apply(lambda x: len(str(x).split()))

    lh['to_len'] = lh['to_1_len'] + lh['to_2_len']
    lh['from_len'] = lh['from_1_len'] + lh['from_2_len']

    lh['total_len'] = lh['to_len'] + lh['from_len']

    lh.reset_index(inplace=True, drop=True)
    
    print(len(lh))

    other_issues = []
    for i in lh.index:
        if lh['to_1_len'].iloc[i]==1:
            other_issues.append(lh['to'].iloc[i])
        elif lh['from_1_len'].iloc[i]==1:
            other_issues.append(lh['from'].iloc[i])
        elif lh['to_2_len'].iloc[i]==1:
            other_issues.append(lh['other_issue_to'].iloc[i])
        elif lh['from_2_len'].iloc[i]==1:
            other_issues.append(lh['other_issue_from'].iloc[i])
        elif lh['to_len'].iloc[i]>0:
            match = re.search('[A-Z]+-[0-9]+', str(lh['to'].iloc[i]))
            other_issues.append(lh['to'].iloc[i][match.start():match.end()])
        elif lh['from_len'].iloc[i]>0:
            match = re.search('[A-Z]+-[0-9]+', str(lh['from'].iloc[i]))
            other_issues.append(lh['to'].iloc[i][match.start():match.end()])
        else: 
            print(lh.iloc[i])
    
    lh['other_issue_id'] = other_issues
    
    link_hist_dict[s] = lh

In [ ]:
maintainer_teams = {
    'Repo': SOURCES
}

overview = pd.DataFrame(maintainer_teams)

for pre in [0]:
    num_components = []
    for s in SOURCES:
#         print(s)
        temp = link_hist_dict_2[s]
        
        project_df = issue_dict[s][['issue_id', 'projectid']]

        lh_plus = temp.merge(project_df, left_on='issue_id', right_on='issue_id')
        lh_plus = lh_plus.merge(project_df, left_on='other_issue_id', right_on='issue_id', suffixes=('_iid', '_oid'))
        
        lh_plus['same_project'] = lh_plus['projectid_iid'] == lh_plus['projectid_oid']
        lh_plus = lh_plus[lh_plus['same_project']==False]
        
        stat_changes = (lh_plus[['projectid_iid', 'projectid_oid']].value_counts()>=len(lh_plus)*pre).reset_index(name='valid')
        stat_changes = stat_changes[stat_changes['valid']==True]
        edge_weights = (lh_plus[['projectid_iid', 'projectid_oid']].value_counts()/lh_plus[['projectid_iid', 'projectid_oid']].value_counts().sum()*25)[0:len(stat_changes)].to_frame().reset_index()

        nodes = set(stat_changes['projectid_iid'].values).union(set(stat_changes['projectid_oid']))                                 

        G = nx.Graph()
        G.add_nodes_from(nodes)

        ew = []
        for i in edge_weights.index:
            from_node = edge_weights.iloc[i]['projectid_iid']
            to_node = edge_weights.iloc[i]['projectid_oid']
            weight = edge_weights.iloc[i][0]
            ew.append(weight)
            G.add_edge(from_node, to_node, weight=weight)

        num_comps = len([len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)])
        num_components.append(num_comps)
        comps = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    #         components.append(comps)
        
        pos = nx.circular_layout(G)

        fig, ax = plt.subplots(figsize=(7, 7))

        nx.draw_networkx_edges(G, pos, alpha=1, width=ew, edge_color="black")
        nx.draw_networkx_nodes(G, pos, node_color="#BB00AA", alpha=1)
        label_options = {"ec": "k", "fc": "white", "alpha": 0.7}
        nx.draw_networkx_labels(G, pos, font_size=14, bbox=label_options)
        # nx.draw_networkx(nx.planar_layout(G))

        font = {"fontname": "Helvetica", "color": "k", "fontweight": "bold", "fontsize": 14}
        ax.set_title(s+ " Projects", font)

        fig.tight_layout()
        plt.axis("off")
        plt.show()
    overview['comps_'+str(pre)] = num_components

    

In [ ]:
overview.to_csv('properties/project_overview.csv', encoding='utf-8', index=True, sep=',')

overview

In [ ]:
link_hist_dict_5 = {}
link_hist_dict_2 = {}

for s in SOURCES:
    print(s)
    temp_df = link_hist_dict[s]
    temp_p = project_activity[project_activity['Repository']==s]
    
    temp_5_df = temp_df[temp_df['year']>=temp_p['last year active'].max()-5]
    temp_2_df = temp_df[temp_df['year']>=temp_p['last year active'].max()-2]
    
    link_hist_dict_5[s] = temp_5_df
    link_hist_dict_2[s] = temp_2_df

In [ ]:
def find_percentage(temp_df, col, perc):
    total = len(temp_df)
    for i in np.linspace(0, 0.3, num=1001):
        k = int(np.round(i*len(temp_df),0))
        x = temp_df[col].value_counts().sort_values(ascending=False)[0:k].sum()/total
        m = (temp_df[col].value_counts().sort_values(ascending=False)[0:k].index)
        if (x >= perc):
            return i, x, m

maintainers_dict = {}
        
for s in SOURCES:
    print(s)
    temp_df = link_hist_dict_2[s]
    
    i, x, m = find_percentage(temp_df, 'maintainer', 0.9)
    print([i, x, len(m)])
    maintainers_dict[s] = m

In [ ]:
maintainer_project_dict = {}
mp_df_dict = {}

for s in SOURCES:
    print(s)
    temp_df = link_hist_dict_2[s]
    temp_df['other_issue_to'].fillna('', inplace=True)
    temp_df['other_issue_from'].fillna('', inplace=True)
    temp_df['other_issue_id'] = temp_df['other_issue_to']+temp_df['other_issue_from']
    temp_df['project_1'] = temp_df['issue_id'].str.extract(r'(\D+)')
    temp_df['project_2'] = temp_df['other_issue_id'].str.extract(r'(\D+)')
    valid_maintainers = maintainers_dict[s]
    temp_df = temp_df[temp_df['maintainer'].isin(valid_maintainers)]
    temp_df_1 = temp_df[['maintainer', 'project_1']]
    temp_df_1.columns = ['maintainer', 'project']
    temp_df_2 = temp_df[['maintainer', 'project_2']]
    temp_df_2.columns = ['maintainer', 'project']
    temp_df_t = pd.concat([temp_df_1, temp_df_2], ignore_index=True)
    
    mp_df_dict[s] = temp_df_t
    
    temp_p = pd.crosstab(index=temp_df_t['maintainer'], columns=temp_df_t['project'])
    temp_p.fillna(0, inplace=True)
    print(len(temp_p)==len(valid_maintainers))
    
    maintainer_project_dict[s] = temp_p

In [ ]:
maintainer_teams = {
    'Repo': SOURCES
}

overview = pd.DataFrame(maintainer_teams)

for i in np.linspace(0.05, 0.25, num=5):
    num_components = []
    for s in SOURCES:
        m_p_df = maintainer_project_dict[s]
        m_p_per_df = m_p_df.div(m_p_df.sum(axis=1), axis=0)
        edges = m_p_per_df[m_p_per_df >= i].stack().index.tolist()
        G = nx.Graph()
        G.add_nodes_from(m_p_df.index, bipartite=0)
        G.add_nodes_from(m_p_df.columns, bipartite=1)
        G.add_edges_from(edges)
        num_comps = len([len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True) if len(c)>1])
        num_components.append(num_comps)
        comps = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True) if len(c)>1]
#         components.append(comps)
    overview['comps_'+str(round(i,2))] = num_components

In [ ]:
overview.set_index('Repo', inplace=True)
overview.to_csv('properties/maintainer_overview.csv', encoding='utf-8', index=True, sep=',')

overview

In [ ]:
maintainer_teams = {
    'Repo': SOURCES
}

overview = pd.DataFrame(maintainer_teams)

for i in np.linspace(0.05, 0.25, num=5):
    num_components = []
    for s in SOURCES:
        temp = link_hist_dict[s]
        project_df = issue_dict[s][['issue_id', 'projectid']]

        lh_plus = temp.merge(project_df, left_on='issue_id', right_on='issue_id')
        lh_plus = lh_plus.merge(project_df, left_on='other_issue_id', right_on='issue_id', suffixes=('_iid', '_oid'))

        temp = lh_plus[['projectid_iid', 'projectid_oid']].value_counts()
        edges = m_p_per_df[m_p_per_df >= i].stack().index.tolist()
        G = nx.Graph()
        G.add_nodes_from(temp.index)
        G.add_edges_from(edges)
        num_comps = len([len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True) if len(c)>1])
        num_components.append(num_comps)
        comps = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True) if len(c)>1]
#         components.append(comps)
    overview['comps_'+str(round(i,2))] = num_components

